In [106]:
import pandas as pd

train_data = pd.read_csv("kg_train.csv/kg_train.csv",encoding='latin-1')

In [107]:
import string
print(string.punctuation)
specialchars = list(string.punctuation)
print(specialchars)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']


In [108]:
import re
from spellchecker import SpellChecker
from textblob import TextBlob

import nltk
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer

patterns = [
re.compile(r'\W'),
re.compile(r'\s+[a-zA-Z]\s+'),
re.compile(r'\s*([0-9])+\s*'),
re.compile(r'\^[a-zA-Z]\s+'),
re.compile(r'\s+'),
re.compile(r'^b\s+')]

wordnet_lemma  = WordNetLemmatizer()

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_text(doc):
    list_words = []
    for item in doc.split():
        list_words.append(wordnet_lemma.lemmatize(item,get_wordnet_pos(item)))
    return(' '.join(map(str, list_words)))

def clean_text(text):
    
    #Remove all the special characters
    processed_feature = re.sub(patterns[0], ' ', str(text))

    # remove all single characters
    processed_feature= re.sub(patterns[1], ' ', processed_feature)
    
    # remove all single numbers, or all numbers attached to a string
    processed_feature= re.sub(patterns[2], ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(patterns[3], ' ', processed_feature) 

    # Substituting multiple spaces with single space
    processed_feature = re.sub(patterns[4], ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'
    processed_feature = re.sub(patterns[5], '', processed_feature)
    
    ######### Correct spelling and grammar: Using Textblob
    #blob = TextBlob(processed_feature)
    #processed_feature = str(blob.correct())

    # Converting to Lowercase
    processed_feature = processed_feature.lower()
    
    return lemmatize_text(processed_feature)

In [ ]:
train_data['processed_text'] = train_data['text'].apply(clean_text)

In [113]:
train_data

,text,label,processed_text
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,dear sir strictly private business proposal be...
1,Will do.,0,will do
2,Nora--Cheryl has emailed dozens of memos about...,0,nora cheryl have email dozen of memo about hai...
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,dear sir fmadam c know that this proposal migh...
4,fyi,0,fyi
...,...,...,...
5959,I talked to CDM about doing some strategy sess...,0,i talk to cdm about do some strategy session t...
5960,"Dear Sir/CEO,=20I am Mr.D S Ammer, the purchas...",1,dear sir ceo i be mr s ammer the purchasing di...
5961,Faxing to you now.,0,fax to you now
5962,"Goodday,I am Joseph vaye,the son of late Issac...",1,goodday be joseph vaye the son of late issac n...


In [114]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

sub_data_train, sub_data_val, sub_label_train, sub_label_val = train_test_split(train_data, train_data["label"], test_size=0.2, random_state=5)

bow_transformer = CountVectorizer().fit(sub_data_train['processed_text'])

X_train = bow_transformer.transform(sub_data_train['processed_text'])
X_val  = bow_transformer.transform(sub_data_val['processed_text'])

print(X_train.shape)
print(X_val.shape)

sub_data_train['text']

(4771, 78655)
(1193, 78655)


4159    http://www.quinnipiac.edu/x1284.xml?ReleaselD=...
2426                         Dear Sir, It is my great ...
2446    Yes - will call later tonite or am.Had b/fast ...
1496    PLEASE,DO RECONFIRM TO THIS OFFICE ,AS A MATTE...
3662    Mr.SAMUEL PETTERSBOA Bank of AfricaBurkina Fas...
                              ...                        
3046                We have itAlready working on schedule
1725    Abedin Huma <AbedinHÃÂ©state.gov>Friday May 2...
4079    5 RIDER HAGGARD CLOSE=2C JO=2C BORG SOUTH AFRI...
2254    Some cats and dogs:1. Talked to Denis about EU...
2915    Mills Cheryl D <MillsCD@state.gov>Monday Janua...
Name: text, Length: 4771, dtype: object

In [115]:
#Learn Classifier
clf = MultinomialNB().fit(X_train, sub_label_train)
#Predict Val data
pred_val = clf.predict(X_val)

accuracy = accuracy_score(sub_label_val,pred_val)
print(accuracy)
confusion_matrix(sub_label_val, pred_val)

0.9681475272422464


array([[641,  37],
       [  1, 514]])

In [ ]:
data_test = pd.read_csv("kg_test.csv/kg_test.csv",encoding='latin-1')
X_test = bow_transformer.transform(data_test['text'].apply(clean_text))
pred_text = clf.predict(X_test)
submission_file = pd.DataFrame({'Id': data_test.index,'Category':pred_text})
submission_file.to_csv('to_submit.csv',index=False)

# To do's

In [ ]:
#FIRST: Lemmatizing (DONE)

In [ ]:
#SECOND: Spelling and grammar correction

In [ ]:
#THIRD: Implement formula to compute efficiency-accuracy ratio

In [ ]:
#FOURTH: Weird a's show up in dictionary